# 네이버 API에서 키워드를 입력받아 블로그, 뉴스, 책 데이터 수집
### 각각의 csv 파일로 저장
### 파일명에는 수집 날짜가 자동으로 파일명에 포함되도록(데이트타임 문자열)
### 반복되는 부분은 함수를 이용해서 중복 줄이기

In [7]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path="./data/.env_naver")
import requests

total_page=1
page=1
num=1
category=['blog','news','book']

blog=[]
news=[]
book=[]
keyword=input("키워드를 입력하세요.")
while page<=total_page:
    user_id=os.getenv("user_Id")
    user_secret=os.getenv("user_Secret")
    
    for i in category:
        url=f"https://openapi.naver.com/v1/search/{i}"
        headers={"X-Naver-Client-Id":user_id,"X-Naver-Client-Secret" :user_secret}
        payload = dict(query=keyword,display=100,start=num,sort="date")
        r=requests.get(url,params=payload, headers=headers)
        print(r.url)
        print(r.status_code)
        response= r.json()
        if i=='blog':
            blog.append(response['items'])
            if response['total']//100>11:
                total_page=11
            else:
                total_page=response['total']//100+1
        elif i=='news':
            news.append(response['items'])
        else:
            book.append(response['items'])

            
            

            # 페이지 증가
            page+=1

            #num이 10000이 되도록
            if num<901:
                num+=100
            elif num>=901:
                num+=99

키워드를 입력하세요.핀테크
https://openapi.naver.com/v1/search/blog?query=%ED%95%80%ED%85%8C%ED%81%AC&display=100&start=1&sort=date
200
https://openapi.naver.com/v1/search/news?query=%ED%95%80%ED%85%8C%ED%81%AC&display=100&start=1&sort=date
200
https://openapi.naver.com/v1/search/book?query=%ED%95%80%ED%85%8C%ED%81%AC&display=100&start=1&sort=date
200
https://openapi.naver.com/v1/search/blog?query=%ED%95%80%ED%85%8C%ED%81%AC&display=100&start=101&sort=date
200
https://openapi.naver.com/v1/search/news?query=%ED%95%80%ED%85%8C%ED%81%AC&display=100&start=101&sort=date
200
https://openapi.naver.com/v1/search/book?query=%ED%95%80%ED%85%8C%ED%81%AC&display=100&start=101&sort=date
200
https://openapi.naver.com/v1/search/blog?query=%ED%95%80%ED%85%8C%ED%81%AC&display=100&start=201&sort=date
200
https://openapi.naver.com/v1/search/news?query=%ED%95%80%ED%85%8C%ED%81%AC&display=100&start=201&sort=date
200
https://openapi.naver.com/v1/search/book?query=%ED%95%80%ED%85%8C%ED%81%AC&display=100&start=201&sort=d

In [8]:
import re
def text_clean(text):
    # html 태그를 없애는 정규표현식
    result =re.sub(r"</?[^>]+>","",text) 
    # 한글, 영문, 숫자 외의 모든 문자 제거 후 공백으로 변환
    result=re.sub(r"[^가-힣a-zA-Z0-9]"," ",result)
    result=result.replace("  "," ").replace("  "," ").replace("  "," ")
    return result

In [9]:
import pandas as pd

In [10]:
def data_frame(categ):
    result={}
    for page in categ:
        for item in page:
            for key, value in item.items():
                if key in ["title","description"]:
                    value=text_clean(value)
                    result.setdefault(key,[]).append(text_clean(value))
                else:
                    result.setdefault(key,[]).append(value)
    df=pd.DataFrame(result)
    return df

In [11]:
data_frame(blog)

,title,link,description,bloggername,bloggerlink,postdate
0,경제 금산분리 금융과 산업 어디까지 함께할,https://blog.naver.com/hyoline1987/224042023830,금산분리 완화 vs 강화 최근에는 빅테크 플랫폼 기업의 금융 진출 핀테크 산업의 성...,눈코입 그리고 귀,blog.naver.com/hyoline1987,20251015
1,신한카드 장기카드대출 신청조건과 상환방법 정리,https://think-tank.tistory.com/888,그리고 핀테크 기업의 약진이라는 삼중고를 겪었습니다 고금리 시대에 접어들면서 카드론...,think-tank.tistory.com,https://think-tank.tistory.com/,20251015
2,스테이블코인의 진화,https://blog.naver.com/skehlsek0103/224042019594,써클 USDC 규제 친화형 디지털 달러 써클 Circle 은 2013년 미국 매사추...,사부아 비부르 삶을 꿈꾸다,blog.naver.com/skehlsek0103,20251015
3,루민트 상장 이후 금융 시장은 어떻게 반응할까,https://blog.naver.com/jskjs1119/224042020394,금융권의 관심 루민트는 상장 전임에도 국내외 자산운용사 핀테크 스타트업 일부 기관투...,깅탱경 님의 블로그,blog.naver.com/jskjs1119,20251015
4,리얼리뷰 사기 사칭 부업팀미션 속지 말아야 할 이유,https://blog.naver.com/mgreatmaild18/224042019913,이유 팀미션사기 주식사기 코인사기 스캠사기 사칭사기 공모주사기 투자사기 투자사기대응...,daylaw,blog.naver.com/mgreatmaild18,20251015
...,...,...,...,...,...,...
1095,2030세대 주목 리플 XRP 코인 미래 재테크의 치트키가 될까,https://blog.naver.com/deanhan84/224038338391,리플랩스 Ripple Labs 라는 핀테크 기업이 만든 이 디지털 자산은 국제 송금...,건강 재테크 연구소,blog.naver.com/deanhan84,20251012
1096,미래를 여는 열쇠 핀테크 재산 분할 그리고 정의로운 법의 역할,https://blog.naver.com/iwvygfbhiu/224038338804,미래를 여는 열쇠 핀테크 재산 분할 그리고 정의로운 법의 역할 미래를 여는 열쇠 핀...,건강이와 맛있는 오늘 ♡,blog.naver.com/iwvygfbhiu,20251012
1097,김병주 MBK 회장과 오경석 업비트 대표 국감 증인 소환 이유는,https://blog.naver.com/beautifuldays6512/22403...,또한 네이버가 두나무를 자회사로 편입하는 방안도 큰 관심을 받고 있으며 이는 국내 ...,beautifuldays6512님의 블로그,blog.naver.com/beautifuldays6512,20251012
1098,블록체인 비저너리2 TOKEN2049 PayFi Summit 국경 없는,https://blog.naver.com/goodblock25/224038334285,동남아시아의 유니콘 핀테크 기업들이 과거에는 상상도 못했지만 이제는 스테이블코인 결...,goodblock25,blog.naver.com/goodblock25,20251012


In [103]:
from datetime import datetime

def csvfile(df,categ):
    today = datetime.now().strftime("%Y-%m-%d")
    df.to_csv(f"./data/{categ}데이터수집_{today}.csv",index=False,encoding="utf-8-sig")

In [104]:
csvfile(data_frame(blog),"blog")

In [105]:
csvfile(data_frame(news),"news")

In [106]:
csvfile(data_frame(book),"book")